# Hackingforfuture Task 4 - Predictive Quality Data Analysis

some details on this hackathon... To be filled during the presentation phase.

# Import Libs

In [1]:
import numpy as np
import pandas as pd
import feature_engine as fe

# Loading file
The file in our case is transformed into csv-file.

In [2]:
filename = 'csvdata.csv'
df = pd.read_csv(filename)
df.head()

,Time,Date,asm_Force #D,asm_Length #E,asm_Height #F,asm_Press-fit #G,asm_force #H,asm_Length #I,asm_Depth #J,asm_depth #K,...,"fin_Tracking, Zone3 #BU","fin_Tracking, Zone4 #BV","fin_Tracking, Zone1, G #BW","fin_Tracking, Zone2, G #BX","fin_Tracking, Zone3, G #BY","fin_Tracking, Zone4, G #BZ",fin_Shape #CA,fin_pressure #CB,fin_flow #CC,fin_temperature #CD
0,02:02:09,03.01.18,0.149,102.52,0.215,12.747,1.094,135.43,1.8541,1.8517,...,96000.0,139700.0,0.14,0.12,0.07,0.08,V-shape,2.915,0.828,88.15
1,02:02:41,03.01.18,0.163,102.61,0.191,12.731,1.116,135.48,1.8583,1.8556,...,97800.0,138100.0,0.16,0.14,0.06,0.12,V-shape,2.933,1.068,89.14
2,02:05:06,03.01.18,0.160,102.56,0.192,12.734,1.033,135.48,1.8533,1.8487,...,112100.0,140000.0,0.04,0.03,0.02,0.15,V-shape,2.916,0.828,87.30
3,02:05:31,03.01.18,0.158,102.47,0.195,12.726,1.079,135.48,1.8578,1.8546,...,96000.0,140000.0,0.25,0.20,0.07,0.18,Ascending,2.925,1.068,88.68
4,02:08:00,03.01.18,0.154,102.61,0.207,12.752,1.061,135.48,1.8594,1.8543,...,100500.0,137700.0,0.14,0.13,0.10,0.23,Ascending,2.904,0.816,87.29


## Distinguish columns
Due to my laziness to join data table, I created one universal dataframe with prepended column names: `asm_`, `int_`, and `fin_` which stand for assembly data, initial inspection data, and final inspection data respectively.

In [3]:
cols = df.columns.tolist()
# remove # from column names
for idc, c in enumerate(cols):
    for idx, char in enumerate(c):
        if char == '#':
            break
    cols[idc] = c[:idx-1]
cols

['Ti',
 'Da',
 'asm_Force',
 'asm_Length',
 'asm_Height',
 'asm_Press-fit',
 'asm_force',
 'asm_Length',
 'asm_Depth',
 'asm_depth',
 'asm_Depth',
 'asm_parallelism',
 'asm_force',
 'asm_Length',
 'asm_force',
 'asm_Length',
 'asm_torque 1',
 'asm_angle 1',
 'asm_torque 2',
 'asm_angle 2',
 'asm_torque 3',
 'asm_angle 3',
 'asm_torque 1',
 'asm_torque 2',
 'asm_torque 3',
 'asm_angle 1',
 'asm_angle 2',
 'asm_angle 3',
 'asm_torque 1',
 'asm_torque 2',
 'asm_torque 3',
 'asm_torque 4',
 'asm_angle 1',
 'asm_angle 2',
 'asm_angle 3',
 'asm_angle 4',
 'asm_temperature',
 'asm_force',
 'asm_Length',
 'asm_force',
 'asm_Temperature',
 'asm_angle',
 'asm_torque',
 'asm_angle',
 'asm_torque',
 'asm_clearance',
 'int_weight',
 'int_Angle position ',
 'int_weight',
 'int_Angle position',
 'int_Tracking, Zone1',
 'int_Tracking, Zone2',
 'int_Tracking, Zone3',
 'int_Tracking, Zone4',
 'int_Tracking, Zone1, G',
 'int_Tracking, Zone2, G',
 'int_Tracking, Zone3, G',
 'int_Tracking, Zone4, G',
 'int

In [18]:
def redefine_cols(cols):
    cols_asm = [c for c in cols if 'asm_' == c[:4]]
    cols_int = [c for c in cols if 'int_' == c[:4]]
    cols_fin = [c for c in cols if 'fin_' == c[:4]]
    return cols_asm, cols_int, cols_fin
cols_asm, cols_int, cols_fin = redefine_cols(cols)
print("Assembly data columns: {}".format(len(cols_asm)))
print("Initial inspection data columns: {}".format(len(cols_int)))
print("Final inspection data columns: {}".format(len(cols_fin)))

Assembly data columns: 44
Initial inspection data columns: 16
Final inspection data columns: 16


In [5]:
df.columns = cols

# Exploratory Data Analysis
This part involves EDA process. The first step is to find Nan.

In [6]:
# Find Nan
df.isnull().mean()

Ti                        0.000000
Da                        0.000000
asm_Force                 0.000000
asm_Length                0.000000
asm_Height                0.000000
                            ...   
fin_Tracking, Zone4, G    0.042976
fin_Shape                 0.042976
fin_pressure              0.042976
fin_flow                  0.042976
fin_temperature           0.042976
Length: 78, dtype: float64

There are around 5% of Nan in this data set. From the readme file, this is due to quality stuffs or whatever. The Nan are only in the final inspection.

In [7]:
# Check data type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16963 entries, 0 to 16962
Data columns (total 78 columns):
Ti                        16963 non-null object
Da                        16963 non-null object
asm_Force                 16963 non-null float64
asm_Length                16963 non-null float64
asm_Height                16963 non-null float64
asm_Press-fit             16963 non-null float64
asm_force                 16963 non-null float64
asm_Length                16963 non-null float64
asm_Depth                 16963 non-null float64
asm_depth                 16963 non-null float64
asm_Depth                 16963 non-null float64
asm_parallelism           16963 non-null float64
asm_force                 16963 non-null float64
asm_Length                16963 non-null float64
asm_force                 16963 non-null float64
asm_Length                16963 non-null float64
asm_torque 1              16963 non-null float64
asm_angle 1               16963 non-null float64
asm_torque 

Most of the data are number variables. There are `Da`, `Ti`, `int_Shape`, and `fin_Shape` which are categorical variables. These variables shall be encoded afterwards.

In [8]:
# After this point will be data processing. The initial dataframe shall be backed up first.
df_bkup = df

# Nan processing
This section dedicates for Nan processing. The quick-dirty process is to remove the Nan out. If there is time left, data imputation can be done.

In [9]:
# remove nan
df = df.dropna()
df.isnull().mean()

Ti                        0.0
Da                        0.0
asm_Force                 0.0
asm_Length                0.0
asm_Height                0.0
                         ... 
fin_Tracking, Zone4, G    0.0
fin_Shape                 0.0
fin_pressure              0.0
fin_flow                  0.0
fin_temperature           0.0
Length: 78, dtype: float64

# Train-test split
Do traintestsplit here to avoid information leakage.

# Transform categorical data to numerical

In [10]:
# distinguish between categorical and numerical var
catcols = ['int_Shape', 'fin_Shape']
numcols = [c for c in cols[2:] if c not in catcols]
numcols

['asm_Force',
 'asm_Length',
 'asm_Height',
 'asm_Press-fit',
 'asm_force',
 'asm_Length',
 'asm_Depth',
 'asm_depth',
 'asm_Depth',
 'asm_parallelism',
 'asm_force',
 'asm_Length',
 'asm_force',
 'asm_Length',
 'asm_torque 1',
 'asm_angle 1',
 'asm_torque 2',
 'asm_angle 2',
 'asm_torque 3',
 'asm_angle 3',
 'asm_torque 1',
 'asm_torque 2',
 'asm_torque 3',
 'asm_angle 1',
 'asm_angle 2',
 'asm_angle 3',
 'asm_torque 1',
 'asm_torque 2',
 'asm_torque 3',
 'asm_torque 4',
 'asm_angle 1',
 'asm_angle 2',
 'asm_angle 3',
 'asm_angle 4',
 'asm_temperature',
 'asm_force',
 'asm_Length',
 'asm_force',
 'asm_Temperature',
 'asm_angle',
 'asm_torque',
 'asm_angle',
 'asm_torque',
 'asm_clearance',
 'int_weight',
 'int_Angle position ',
 'int_weight',
 'int_Angle position',
 'int_Tracking, Zone1',
 'int_Tracking, Zone2',
 'int_Tracking, Zone3',
 'int_Tracking, Zone4',
 'int_Tracking, Zone1, G',
 'int_Tracking, Zone2, G',
 'int_Tracking, Zone3, G',
 'int_Tracking, Zone4, G',
 'int_pressure',
 '

In [11]:
df[catcols[0]].unique()

array(['Ascending', 'V-shape', 'Descending', 'Flat'], dtype=object)

In [12]:
df[catcols[1]].unique()

array(['V-shape', 'Ascending', 'Descending', 'Flat'], dtype=object)

The two categorical variables have identical categories. This means that there is no missing category in both set.

In [17]:
ohe_enc = fe.categorical_encoders.OneHotCategoricalEncoder(
    top_categories=None,
    variables = catcols,
    drop_last=True) # k-1
ohe_enc.fit(df)
df = ohe_enc.transform(df)
df.head()

,Ti,Da,asm_Force,asm_Length,asm_Height,asm_Press-fit,asm_force,asm_Length,asm_Depth,asm_depth,...,"fin_Tracking, Zone4, G",fin_pressure,fin_flow,fin_temperature,int_Shape_Ascending,int_Shape_V-shape,int_Shape_Descending,fin_Shape_V-shape,fin_Shape_Ascending,fin_Shape_Descending
0,02:02:09,03.01.18,0.149,102.52,0.215,12.747,1.094,135.43,1.8541,1.8517,...,0.08,2.915,0.828,88.15,1,0,0,1,0,0
1,02:02:41,03.01.18,0.163,102.61,0.191,12.731,1.116,135.48,1.8583,1.8556,...,0.12,2.933,1.068,89.14,0,1,0,1,0,0
2,02:05:06,03.01.18,0.160,102.56,0.192,12.734,1.033,135.48,1.8533,1.8487,...,0.15,2.916,0.828,87.30,0,1,0,1,0,0
3,02:05:31,03.01.18,0.158,102.47,0.195,12.726,1.079,135.48,1.8578,1.8546,...,0.18,2.925,1.068,88.68,0,1,0,0,1,0
4,02:08:00,03.01.18,0.154,102.61,0.207,12.752,1.061,135.48,1.8594,1.8543,...,0.23,2.904,0.816,87.29,0,0,1,0,1,0


In [23]:
# After modifying the columns, redefine the column names
cols_asm, cols_int, cols_fin = redefine_cols(df.columns)
print("Number of initial inspection columns atfter OneHotEncoding: ", len(cols_int))
print("Number of final inspection columns atfter OneHotEncoding: ", len(cols_fin))

Number of initial inspection columns atfter OneHotEncoding:  18
Number of final inspection columns atfter OneHotEncoding:  18


# Scale the numerical data

# Discretize the output 